# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [187]:
from datetime import datetime, date, timedelta, timezone
from scipy.stats import norm

import yfinance as yf
import pandas as pd
import numpy as np
from typing import List, Tuple

# Exploration

In [3]:
msft = yf.Ticker("MSFT")

# get all stock info
# msft.info

# get historical market data
# hist = msft.history(period="1mo")

# show meta information about the history (requires history() to be called first)
# msft.history_metadata

# show actions (dividends, splits, capital gains)
# msft.actions
# msft.dividends
# msft.splits
# msft.capital_gains  # only for mutual funds & etfs

# show share count
# msft.get_shares_full(start="2024-01-01", end=None)

# show financials:
# # - income statement
# msft.income_stmt
# msft.quarterly_income_stmt
# # - balance sheet
# msft.balance_sheet
# msft.quarterly_balance_sheet
# # - cash flow statement
# msft.cashflow
# msft.quarterly_cashflow
# # see `Ticker.get_income_stmt()` for more options

# show holders
# msft.major_holders
# msft.institutional_holders
# msft.mutualfund_holders
# msft.insider_transactions
# msft.insider_purchases
# msft.insider_roster_holders

# show recommendations
# msft.recommendations
# msft.recommendations_summary
# msft.upgrades_downgrades

# Show future and historic earnings dates, returns at most next 4 quarters and last 8 quarters by default.
# Note: If more are needed use msft.get_earnings_dates(limit=XX) with increased limit argument.
# msft.earnings_dates

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
# msft.isin

# show options expirations
# msft.options

# show news
# msft.news

In [173]:
# hist

In [174]:
# # get option chain for specific expiration
# opt = msft.option_chain() #'2024-08-16')
# # data available via: opt.calls, opt.puts

In [175]:
# opt.calls

In [176]:
# # Create Ticker object
# ticker = yf.Ticker("AAPL")

# # Get available expiration dates
# expirations = ticker.options

# # Choose an expiration date
# expiration_date = expirations[0]

# # Fetch option chain
# option_chain = ticker.option_chain(expiration_date)

# # Select a specific call option
# call_option = option_chain.calls.iloc[0]
# call_symbol = call_option['contractSymbol']

# # Fetch historical data for the call option
# call_history = yf.Ticker(call_symbol).history(period="1mo")

# # Print historical prices and current open interest
# print("Historical prices for", call_symbol)
# print(call_history[['Open', 'High', 'Low', 'Close', 'Volume']])
# print("\nCurrent Open Interest:", call_option['openInterest'])

# Options

In [209]:
def weighted_average(values: List[float], weights: List[float]) -> float:
    return round(np.average(values, weights=weights), 4) if len(values) == len(weights) else np.nan

def get_stock_data(tickers: List[str]) -> pd.DataFrame:
    data = []
    for t in tickers:
        try:
            ticker = yf.Ticker(t)
            
            # Get shares outstanding
            shares_outstanding = ticker.info.get('sharesOutstanding', np.nan)
            
            # Get options data
            options_data = get_options_data(ticker)
            if options_data is None:
                continue
            
            oi_calls, oi_puts, iv = options_data
            oi_total = oi_calls + oi_puts
            
            # Calculate ratio
            ratio = round((oi_total * 100) / shares_outstanding, 4) if shares_outstanding else np.nan
            
            # Get company name
            company_name = ticker.info.get('longName', 'N/A')
            
            # Get next earnings date
            next_earnings = get_next_earnings_date(ticker)
            
            # Create DataFrame
            df = pd.DataFrame({
                'ticker': [t],
                'name': [company_name],
                'shares': [shares_outstanding],
                'calls_oi': [oi_calls],
                'puts_oi': [oi_puts],
                'total_oi': [oi_total],
                'ratio': [ratio],
                'iv': [iv],
                'next_earnings': [next_earnings]
            })
            
            data.append(df)
        except Exception as e:
            print(f"Error processing {t}: {str(e)}")
            continue
    
    return pd.concat(data, ignore_index=True) if data else pd.DataFrame()

def get_options_data(ticker: yf.Ticker) -> Tuple[int, int, float]:
    try:
        options = ticker.options
        oi_calls, oi_puts = 0, 0
        iv_weighted_sum, total_weight = 0, 0
        
        for date in options:
            opt = ticker.option_chain(date)
            
            calls_oi = opt.calls['openInterest'].fillna(0).astype(int)
            puts_oi = opt.puts['openInterest'].fillna(0).astype(int)
            
            oi_calls += calls_oi.sum()
            oi_puts += puts_oi.sum()
            
            calls_iv = opt.calls['impliedVolatility'].fillna(0)
            puts_iv = opt.puts['impliedVolatility'].fillna(0)
            
            iv_weighted_sum += (calls_iv * calls_oi).sum() + (puts_iv * puts_oi).sum()
            total_weight += calls_oi.sum() + puts_oi.sum()
        
        iv = round(iv_weighted_sum / total_weight, 4) if total_weight > 0 else np.nan
        
        return oi_calls, oi_puts, iv
    except Exception as e:
        print(f"Error fetching options data: {str(e)}")
        return None

def get_next_earnings_date(ticker: yf.Ticker) -> str:
    try:
        earnings_calendar = ticker.earnings_dates
        if earnings_calendar is not None and not earnings_calendar.empty:
            today = datetime.now(timezone.utc).date()
            future_dates = earnings_calendar[earnings_calendar.index.date > today]
            if not future_dates.empty:
                next_date = future_dates.index[-1]
                return next_date.strftime('%Y-%m-%d')
            else:
                return 'N/A'  # 'No future earnings date available'
    except Exception as e:
        pass
        # print(f"Error fetching earnings date: {str(e)}")
    return 'N/A'

In [210]:
%%time

# Usage
tickers = ['NVDA', 'AAPL', 'MSFT']#, 'GOOGL', 'AMZN']  # Add more tickers as needed
result = get_stock_data(tickers)

CPU times: user 1.9 s, sys: 102 ms, total: 2.01 s
Wall time: 7.34 s


In [211]:
result.head()

,ticker,name,shares,calls_oi,puts_oi,total_oi,ratio,iv,next_earnings
0,NVDA,NVIDIA Corporation,24598300672,13770351,13660368,27430719,0.1115,0.7505,2024-08-21
1,AAPL,Apple Inc.,15334099968,3659201,2417840,6077041,0.0396,0.3526,2024-08-01
2,MSFT,Microsoft Corporation,7432309760,1112556,1008079,2120635,0.0285,0.3410,2024-07-30


In [212]:
# Open the file
filepath = 'tickers/nyse_tickers.txt'

with open(filepath, 'r') as file:
    # Read the lines and store them in a list
    lines = file.readlines()

# Remove newline characters
tickers = [line.strip() for line in lines]

# Print the list to verify
print(tickers[:10])
print(len(tickers))

print("est. rutime:", round(len(tickers)*3.8 / 60), "min")

['A', 'AA', 'AACT', 'AAN', 'AAP', 'AAT', 'AB', 'ABBV', 'ABEV', 'ABG']
3170
est. rutime: 116 min


In [213]:
%%time

df = get_stock_data(tickers)

AACT: $AACT: possibly delisted; No earnings dates found
ACP: $ACP: possibly delisted; No earnings dates found
ACP: $ACP: possibly delisted; No earnings dates found
ACV: $ACV: possibly delisted; No earnings dates found
AEFC: $AEFC: possibly delisted; No earnings dates found
AFGB: $AFGB: possibly delisted; No earnings dates found
AFGC: $AFGC: possibly delisted; No earnings dates found
AFGD: $AFGD: possibly delisted; No earnings dates found
AFGE: $AFGE: possibly delisted; No earnings dates found
AFT: $AFT: possibly delisted; No earnings dates found
AGD: $AGD: possibly delisted; No earnings dates found


Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Ear

ANG: $ANG: possibly delisted; No earnings dates found
ANG: $ANG: possibly delisted; No earnings dates found
B: $B: possibly delisted; No earnings dates found


Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Ear

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ARGO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ARGO&crumb=wnkN2eKlIHN


...

Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: 'Earnings Date'
Error fetching earnings date: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Read timed out. (read timeout=30)
Error processing VTMX: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Error processing VTN: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


VVR: $VVR: possibly delisted; No earnings dates found
WDI: $WDI: possibly delisted; No earnings dates found
WEA: $WEA: possibly delisted; No earnings dates found
WEL: $WEL: possibly delisted; No earnings dates found
Y: $Y: possibly delisted; No earnings dates found
WIA: $WIA: possibly delisted; No earnings dates found
WIW: $WIW: possibly delisted; No earnings dates found


Error fetching earnings date: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Read timed out.
Error processing WSM: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Error processing WSO: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))


XFLT: $XFLT: possibly delisted; No earnings dates found
XFLT: $XFLT: possibly delisted; No earnings dates found
ZTR: $ZTR: possibly delisted; No earnings dates found


CPU times: user 13min 18s, sys: 52.9 s, total: 14min 11s
Wall time: 3h 44min 49s


In [214]:
df.head()

,ticker,name,shares,calls_oi,puts_oi,total_oi,ratio,iv,next_earnings
0,A,"Agilent Technologies, Inc.",291760992.0,7256,6868,14124,0.0048,0.3553,2024-08-21
1,AA,Alcoa Corporation,179560992.0,119870,109514,229384,0.1277,0.5834,2024-10-16
2,AACT,Ares Acquisition Corporation II,50000000.0,0,0,0,0.0000,NaN,N/A
3,AAN,"The Aaron's Company, Inc.",31522300.0,1006,604,1610,0.0051,0.3135,2024-08-05
4,AAP,"Advance Auto Parts, Inc.",59622300.0,43606,57099,100705,0.1689,0.5863,2024-08-21


In [215]:
df.to_csv('data/nyse_stock_data.csv', index=False)

# Options EDA

In [220]:
df.drop_duplicates(subset=['ticker'], inplace=True)

In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2442 entries, 0 to 3163
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ticker         2442 non-null   object 
 1   name           2442 non-null   object 
 2   shares         2295 non-null   float64
 3   calls_oi       2442 non-null   int64  
 4   puts_oi        2442 non-null   int64  
 5   total_oi       2442 non-null   int64  
 6   ratio          2295 non-null   float64
 7   iv             1808 non-null   float64
 8   next_earnings  2442 non-null   object 
dtypes: float64(3), int64(3), object(3)
memory usage: 190.8+ KB


In [235]:
df['ratio_to_iv_ratio'] = df.ratio / df.iv

In [232]:
df[df.ratio >= .075].sort_values(['iv']).head(25)

,ticker,name,shares,calls_oi,puts_oi,total_oi,ratio,iv,next_earnings
2985,VLD,"Velo3D, Inc.",8.487570e+06,77391,11321,88712,1.0452,0.1454,2024-08-08
445,BKKT,"Bakkt Holdings, Inc.",6.231820e+06,61686,2154,63840,1.0244,0.1737,N/A
2093,NYCB,"New York Community Bancorp, Inc.",3.513650e+08,235596,285236,520832,0.1482,0.2856,2024-07-25
17,F,Ford Motor Company,3.921490e+09,1582291,2139582,3721873,0.0949,0.3125,2024-07-24
2161,OXY,Occidental Petroleum Corporation,8.866370e+08,390283,500497,890780,0.1005,0.3171,N/A
937,DVN,Devon Energy Corporation,6.275720e+08,265392,225333,490725,0.0782,0.3276,N/A
32,C,Citigroup Inc.,1.925700e+09,1170497,980047,2150544,0.1117,0.3346,2024-10-15
1134,FDX,FedEx Corporation,2.443020e+08,75201,107978,183179,0.0750,0.3489,2024-09-19
2002,NKE,"NIKE, Inc.",1.211460e+09,808712,546032,1354744,0.1118,0.3490,N/A
351,BBY,"Best Buy Co., Inc.",2.157140e+08,100417,65916,166333,0.0771,0.3760,2024-08-27


In [242]:
fltr = (df.ticker != 'BHIL')
df[fltr].sort_values('ratio_to_iv_ratio', ascending=False).head(25).reset_index(drop=True)

# watchlist:
'''
STNG
X


CRWD
DAL (maybe just short) [next earnings in Oct]
FDX (next earnings Sept)

'''

,ticker,name,shares,calls_oi,puts_oi,total_oi,ratio,iv,next_earnings,ratio_to_iv_ratio
0,VLD,"Velo3D, Inc.",8.487570e+06,77391,11321,88712,1.0452,0.1454,2024-08-08,7.188446
1,BKKT,"Bakkt Holdings, Inc.",6.231820e+06,61686,2154,63840,1.0244,0.1737,N/A,5.897524
2,X,United States Steel Corporation,2.248500e+08,360843,717096,1077939,0.4794,0.4696,2024-08-01,1.020869
3,CPRI,Capri Holdings Limited,1.166490e+08,206497,293718,500215,0.4288,0.6695,2024-08-06,0.640478
4,RH,RH,1.844520e+07,31979,38779,70758,0.3836,0.6397,2024-09-05,0.599656
5,SPCE,"Virgin Galactic Holdings, Inc.",2.056810e+07,95324,145508,240832,1.1709,1.9900,2024-07-30,0.588392
6,NYCB,"New York Community Bancorp, Inc.",3.513650e+08,235596,285236,520832,0.1482,0.2856,2024-07-25,0.518908
7,STNG,Scorpio Tankers Inc.,5.393390e+07,86455,37294,123749,0.2294,0.4563,2024-07-30,0.502739
8,AI,"C3.ai, Inc.",1.213180e+08,242093,134395,376488,0.3103,0.6299,N/A,0.492618
9,ZIM,ZIM Integrated Shipping Services Ltd.,1.203210e+08,243422,247386,490808,0.4079,0.8824,2024-08-12,0.462262


In [238]:
df[df.ticker == 'SG']

,ticker,name,shares,calls_oi,puts_oi,total_oi,ratio,iv,next_earnings,ratio_to_iv_ratio
2581,SG,"Sweetgreen, Inc.",100440000.0,31141,25019,56160,0.0559,0.8566,2024-08-08,0.065258


In [239]:
# prob stay away from these where iv is high
df.sort_values('ratio', ascending=False).head(25)

,ticker,name,shares,calls_oi,puts_oi,total_oi,ratio,iv,next_earnings,ratio_to_iv_ratio
2659,SPCE,"Virgin Galactic Holdings, Inc.",20568100.0,95324,145508,240832,1.1709,1.9900,2024-07-30,0.588392
2985,VLD,"Velo3D, Inc.",8487570.0,77391,11321,88712,1.0452,0.1454,2024-08-08,7.188446
445,BKKT,"Bakkt Holdings, Inc.",6231820.0,61686,2154,63840,1.0244,0.1737,N/A,5.897524
2516,SAVE,"Spirit Airlines, Inc.",109501000.0,186297,429718,616015,0.5626,1.4735,N/A,0.381812
3124,X,United States Steel Corporation,224850000.0,360843,717096,1077939,0.4794,0.4696,2024-08-01,1.020869
747,CPRI,Capri Holdings Limited,116649000.0,206497,293718,500215,0.4288,0.6695,2024-08-06,0.640478
3154,ZIM,ZIM Integrated Shipping Services Ltd.,120321000.0,243422,247386,490808,0.4079,0.8824,2024-08-12,0.462262
2428,RH,RH,18445200.0,31979,38779,70758,0.3836,0.6397,2024-09-05,0.599656
422,BIG,"Big Lots, Inc.",29682000.0,45041,64662,109703,0.3696,1.4327,2024-08-27,0.257974
158,AMC,"AMC Entertainment Holdings, Inc.",295588000.0,859551,226174,1085725,0.3673,1.3069,N/A,0.281047


# End